# Artificial Neural Network To Predict The Game Sales In Some Regions

The objetive is to predict the game sales in NA, EU and JP.

Using Video Game Sales dataset of Kaggle.



In [1]:
import pandas as pd
import tensorflow as tf
import sklearn

In [2]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
dataset = pd.read_csv('games.csv')
dataset

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dataset = dataset.drop('Other_Sales', axis= 1)
dataset = dataset.drop('Global_Sales', axis= 1)
dataset = dataset.drop('Developer', axis= 1)

In [6]:
dataset.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [33]:
dataset['Rating'].mode()

0    E
Name: Rating, dtype: object

In [36]:
dataset['User_Score'] = pd.to_numeric(dataset['User_Score'], errors='coerce')

In [39]:
values = {
    'Critic_Score': dataset['Critic_Score'].mean(),
    'Critic_Count': dataset['Critic_Count'].mean(),
    'User_Score': dataset['User_Score'].mean(),
    'User_Count': dataset['User_Count'].mean(),
    'Rating': 'E'
}

dataset = dataset.fillna(value= values)

In [40]:
dataset.isnull().sum()

Name                 2
Platform             0
Year_of_Release    269
Genre                2
Publisher           54
NA_Sales             0
EU_Sales             0
JP_Sales             0
Critic_Score         0
Critic_Count         0
User_Score           0
User_Count           0
Rating               0
dtype: int64

In [44]:
dataset = dataset.dropna(axis= 0)
dataset.shape

(16416, 13)

In [46]:
dataset['Name'].value_counts()

Name
Need for Speed: Most Wanted               12
FIFA 14                                    9
Ratatouille                                9
LEGO Marvel Super Heroes                   9
Monopoly                                   8
                                          ..
Meteos                                     1
Let's Make a Soccer Team!                  1
Loving Life with Hello Kitty & Friends     1
Scrabble (Others sales)                    1
Haitaka no Psychedelica                    1
Name: count, Length: 11397, dtype: int64

In [47]:
dataset = dataset.drop('Name', axis= 1)
dataset.shape

(16416, 12)

In [48]:
x = dataset.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values
y_na = dataset.iloc[:, 4].values
y_eu = dataset.iloc[:, 5].values
y_jp = dataset.iloc[:, 6].values

In [49]:
dataset.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [50]:
# transforming categorical attributes into numeric attributes
onehotencoder = ColumnTransformer(transformers= [("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder= 'passthrough')
x = onehotencoder.fit_transform(x).toarray()

In [52]:
inputLayer = Input(shape= (x.shape[1], ))
hiddenLayer1 = Dense(units= 319, activation= 'relu')(inputLayer)
hiddenLayer2 = Dense(units= 319, activation= 'relu')(hiddenLayer1)
outputLayer1 = Dense(units= 1, activation= 'linear')(hiddenLayer2)
outputLayer2 = Dense(units= 1, activation= 'linear')(hiddenLayer2)
outputLayer3 = Dense(units= 1, activation= 'linear')(hiddenLayer2)

In [53]:
model = Model(inputs= inputLayer, outputs= [outputLayer1, outputLayer2, outputLayer3])

In [54]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate= 0.001), loss= 'mse')

In [55]:
model.fit(
    x,
    [y_na, y_eu, y_jp],
    epochs= 500,
    batch_size= 100
)

Epoch 1/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - dense_2_loss: 196.1567 - dense_3_loss: 190.4803 - dense_4_loss: 435.7057 - loss: 822.3582
Epoch 2/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - dense_2_loss: 2.6402 - dense_3_loss: 0.9489 - dense_4_loss: 0.7791 - loss: 4.3684
Epoch 3/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_2_loss: 2.1579 - dense_3_loss: 0.9288 - dense_4_loss: 0.8069 - loss: 3.8935
Epoch 4/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_2_loss: 7.5473 - dense_3_loss: 7.5324 - dense_4_loss: 8.5994 - loss: 23.6790
Epoch 5/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - dense_2_loss: 4.5464 - dense_3_loss: 1.4152 - dense_4_loss: 1.1627 - loss: 7.1239
Epoch 6/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - dense_2_loss: 19.2431 - dense_3_loss: 8.4166 - dense_4_loss: 16.1742 - loss: 43.8353
Epoch 7/500
165/165 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - dense_2_loss: 5.0184 - dense_3_loss: 2.3387 - dense_4_loss: 1.0759 - loss: 8.4330
Epoch 8/500
165/165 ━━━━━━━━━